## HR Managment Agent

The HR Management Agents acts as the central knowledge and indentity provider for the travel bookiing system. It is responsible for retriving employee information, enforcing traval policy, rules and ensuring that all bookiings comply with company guidelines.

## Setup

Firstly, you are going to install boto3 dependencies from pip. Make sure you have the latest version of it for full capabilities

In [1]:
!pip uninstall boto3 botocore awscli --yes

Found existing installation: boto3 1.38.18
Uninstalling boto3-1.38.18:
  Successfully uninstalled boto3-1.38.18
Found existing installation: botocore 1.38.18
Uninstalling botocore-1.38.18:
  Successfully uninstalled botocore-1.38.18
Found existing installation: awscli 1.40.17
Uninstalling awscli-1.40.17:
  Successfully uninstalled awscli-1.40.17


In [2]:
# Install latest dependencies
!python -m pip install --force-reinstall --no-cache -q -r ../requirements.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.30.0 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.38.18 which is incompatible.
amazon-sagemaker-jupyter-ai-q-developer 1.1.0 requires numpy<=2.0.1, but you have numpy 2.2.6 which is incompatible.
amazon-sagemaker-sql-magic 0.1.4 requires numpy<2, but you have numpy 2.2.6 which is incompatible.
autogluon-common 1.2 requires numpy<2.1.4,>=1.25.0, but you have numpy 2.2.6 which is incompatible.
autogluon-core 1.2

In [3]:
!pip freeze | grep boto3

aioboto3 @ file:///home/conda/feedstock_root/build_artifacts/aioboto3_1742196379442/work
boto3==1.38.18


#### Unique Indetifier

Next, Creating an unique indetifier that will be helpfull to identified all the resources created for hackarthon 

In [6]:
import uuid
import os
from pathlib import Path

def get_or_create_unique_resources_identifier():
    unique_resources_identifier_file = '../.u_resources_identifier'
    
    if os.path.exists(unique_resources_identifier_file):
        with open(unique_resources_identifier_file, 'r') as f:
            return f.read().strip()
    else:
        unique_resources_identifier = str(uuid.uuid4())[:8]
        with open(unique_resources_identifier_file, 'w') as f:
            f.write(unique_resources_identifier)
        return unique_resources_identifier

unique_resources_identifier = get_or_create_unique_resources_identifier()
resource_suffix = f"{unique_resources_identifier}"
print("Your resource suffix is", resource_suffix)

Your resource suffix is 348d2ff0


## Creating Agent

On this section we declare global variables that will be act as helpers during entire notebook and we will start to create your first agent.

In [7]:
import boto3
import os
import json
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]

region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'anthropic.claude-3-5-sonnet-20240620-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0',
    'amazon.titan-embed-text-v2:0',
    'amazon.titan-embed-image-v1',
    'amazon.titan-text-express-v1',
    'amazon.titan-text-lite-v1',
    'ai21.j2-mid-v1',
    'ai21.j2-ultra-v1',
    'cohere.command-text-v14',
    'cohere.embed-english-v3',
    'cohere.embed-multilingual-v3',
    'meta.llama2-13b-chat-v1',
    'meta.llama2-70b-chat-v1',
    'amazon.nova-pro-v1:0'
]

curr_month = datetime.now()

In [51]:
hr_agent_name = f"hr-agent-{resource_suffix}"

hr_lambda_name = f"fn-hr-agent-{resource_suffix}"

hr_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{hr_agent_name}'

dynamodb_table = f"{hr_agent_name}-users"
dynamodb_pk = "emp_id"
dynamodb_sk = 'name'
approval_requests_table = f"{hr_agent_name}-approvals"
approval_pk = "request_id"
approval_sk = "emp_id"
dynamoDB_args = [dynamodb_table, dynamodb_pk, dynamodb_sk]

knowledge_base_name = f'{hr_agent_name}-kb'

knowledge_base_description = "KB containing company travel and HR policies"
bucket_name = f'hr-agent-kb-{account_id}-{resource_suffix}'
print(bucket_name)


hr-agent-kb-225584133776-348d2ff0


### Importing helper functions

On following section, we're adding `bedrock_agent_helper.py` and `knowledge_base_helper` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py` and `knowledge_base_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

On `knowledge_bases.py`:
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [9]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Create and syncronize Knowledge Base

On this section, you're going to create a Amazon Bedrock Knowledge Base and ingest data on it.

This data contains basic information about how forecast process is done.

**This creation process can take several minutes.**

In [10]:
%%time
print(f"Knowledge Base ID: {bucket_name}")
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

Knowledge Base ID: hr-agent-kb-225584133776-348d2ff0
Creating KB hr-agent-348d2ff0-kb
Step 1 - Creating or retrieving hr-agent-kb-225584133776-348d2ff0 S3 bucket for Knowledge Base documents
Creating bucket hr-agent-kb-225584133776-348d2ff0
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_299) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '316',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Mon, 19 May 2025 08:58:20 '
                                                 'GMT',
                                         'x-amzn-requestid': 'd789f4e5-a508-4bf6-af18-0c64e8f2bbd3'},
                        'HTTPStatusCode':

## Create Synthetic Data to Load on S3

Instead of get data elsewhere, you're going to generate data, using a LLM on Amazon Bedrock.
This fake company policy data that will be generated, will be uploaded into a S3 bucket and then added into an Amazon Bedrock Knowledge Base.

In [11]:
path = "kb_documents"

# Check whether the specified path exists or not
is_exist = os.path.exists(path)
if not is_exist:
   # Create a new directory if it does not exist
   os.makedirs(path)
   print("The {} directory was created!".format(path))
else:
   print("The {} directory already exists!".format(path))

The kb_documents directory was created!


Creating helper methods to invoke LLM on Bedrock and to write a local file using Python

In [13]:
def invoke_bedrock_generate_hr_policy_files(prompt):
    message_list = []

    initial_message = {
        "role": "user",
        "content": [
            {
                "text": prompt
            }
        ],
    }

    message_list.append(initial_message)

    response = bedrock_client.converse(
        modelId=agent_foundation_model[0],
        messages=message_list,
        inferenceConfig={
            "maxTokens": 2048,
            "temperature": 0
        },
    )

    return response['output']['message']


def write_file(file_name, content):
    f = open(file_name, 'w')
    f.write(content)
    f.close()

### Generating data prompt
Generating one file with HR travel policy using the LLM model

In [14]:
hr_travel_policy_instructions = '''
    You will act as an HR policy expert specializing in corporate travel management. 
    Create comprehensive documentation for a company's HR and travel policies that will be used 
    by an AI agent to enforce rules and provide information to employees.

    Create the following sections with detailed content:

    1. CORPORATE TRAVEL POLICY
       - Include specific guidelines for booking flights, hotels, and ground transportation
       - Detail approval workflows based on employee grade levels (Junior, Mid-level, Senior, Executive)
       - Specify spending limits for different expense categories by employee grade
       - Include policies for international vs. domestic travel
       - Detail required advance booking timeframes (e.g., 14 days for international)
       - List preferred airline partners and hotel chains
       - Explain reimbursement procedures and required documentation

    2. TRAVEL APPROVAL PROCESS
       - Step-by-step workflow for travel requests
       - Required approvals based on destination, duration, and cost
       - Documentation requirements for different types of travel
       - Emergency travel approval procedures
       - Cancellation and modification policies

    3. EXPENSE REIMBURSEMENT GUIDELINES
       - Eligible and ineligible expenses
       - Per diem allowances by location
       - Receipt requirements
       - Submission deadlines and processing timeframes
       - Corporate card usage policies

    4. EMPLOYEE TRAVEL PROFILES
       - Required documentation (passport, visas, vaccinations)
       - Accommodation preferences and accessibility needs
       - Dietary restrictions handling
       - Emergency contact procedures
       - Duty of care policies

    5. SPECIAL CIRCUMSTANCES
       - Extended stay policies
       - Family/companion travel rules
       - Combined business/personal travel guidelines
       - High-risk destination protocols
       - COVID-19 or health emergency protocols

    Format the content as clear policy statements with numbered sections and subsections.
    Use professional HR language but ensure it's clear and unambiguous.
    Include specific numerical values for all limits, timeframes, and thresholds.
    
    The content will be used by an AI agent to answer employee questions and enforce policy,
    so be comprehensive and precise in your language.
'''

employee_data_schema_instructions = '''
    Create a detailed explanation of the employee data schema used in our DynamoDB table.
    This will be used by the HR agent to understand and query employee information.
    
    The schema includes the following fields:
    
    1. emp_id (String, Primary Key): Employee identification number
    2. name (String): Employee's full name
    3. email (String): Corporate email address
    4. grade (String): Employment grade (Junior, Mid-level, Senior, Executive)
    5. department (String): Department or business unit
    6. manager_id (String): Employee ID of direct manager
    7. nationality (String): Employee's country of citizenship
    8. passport_status (String): Valid, Expiring, or Invalid
    9. passport_expiry (String): Expiration date in YYYY-MM-DD format
    10. preferred_airlines (List): List of preferred airlines
    11. dietary_restrictions (String): Any dietary needs for travel
    12. accessibility_needs (String): Any accessibility accommodations required
    13. emergency_contact (Map): Name and phone number
    14. travel_budget_remaining (Number): Annual travel budget remaining
    15. approval_level (String): Self, Manager, Director, or VP approval required
    
    For each field, explain:
    - The purpose and importance of the field
    - How it's used in travel approval workflows
    - Any validation rules or constraints
    - Examples of valid values
    
    Also include sample queries that the HR agent might need to perform, such as:
    - Looking up an employee by ID or email
    - Checking if an employee has sufficient travel budget
    - Determining the approval chain for a travel request
    - Verifying passport validity for international travel
    
    Format this as a technical document that will help the AI agent understand how to 
    properly access and interpret employee data when handling travel-related inquiries.
'''

hr_policy_files = [
    {'name': 'corporate-travel-policy.txt', 'instructions': hr_travel_policy_instructions},
    {'name': 'employee-data-schema.txt', 'instructions': employee_data_schema_instructions}
]

# Function to generate content and save to files
def generate_hr_policy_files(path):
    for file_info in hr_policy_files:
        print(f"Generating content for {file_info['name']}...")
        
        response_message = invoke_bedrock_generate_hr_policy_files(
            file_info['instructions']
        )
        file_path=''
        content = response_message['content'][0]['text']
        print(f"Generated content for {file_info['name']} (preview):\n{content[:300]}...\n")
        if file_info['name'] == 'employee-data-schema.txt':
            file_path= f"{file_info['name'] }"
        else:   
            file_path = f"{path}/{file_info['name']}"
        write_file(file_path, content)
        print(f"Saved content to {file_path}")

# Create directory for KB documents if it doesn't exist
kb_documents_path = "kb_documents"
if not os.path.exists(kb_documents_path):
    os.makedirs(kb_documents_path)

# Generate the HR policy files
generate_hr_policy_files(kb_documents_path)


Generating content for corporate-travel-policy.txt...
Generated content for corporate-travel-policy.txt (preview):
Here's a comprehensive corporate travel policy document structured as requested:

1. CORPORATE TRAVEL POLICY

1.1 Booking Guidelines
   1.1.1 Flights
      a) All flights must be booked through the company's designated travel management system.
      b) Economy class is standard for all domestic fli...

Saved content to kb_documents/corporate-travel-policy.txt
Generating content for employee-data-schema.txt...
Generated content for employee-data-schema.txt (preview):
Employee Data Schema for Travel Management System

This document outlines the DynamoDB schema for employee data used in our travel management system. It is designed to provide HR agents and AI systems with a comprehensive understanding of ...

Saved content to employee-data-schema.txt


### Uploading data to s3
Uploading generated files into an Amazon S3 Bucket.

In [15]:
#  We can skip user json to upload on s3
def upload_directory(path, bucket_name):
    print('path==='+path)
    print('bucket_name==='+bucket_name)
    for root,dirs,files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root,file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload,bucket_name,file)

### Synchronizing Knowledge Base
Now that the data is available in the s3 bucket, let's synchronize it to our knowledge base

In [16]:
upload_directory("kb_documents", bucket_name)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

uploading file kb_documents/corporate-travel-policy.txt to hr-agent-kb-225584133776-348d2ff0
uploading file kb_documents/.ipynb_checkpoints/corporate-travel-policy-checkpoint.txt to hr-agent-kb-225584133776-348d2ff0
{ 'dataSourceId': 'QBWS4OTNVT',
  'ingestionJobId': 'P6KNUIOXZ8',
  'knowledgeBaseId': 'UVJRQDUOEV',
  'startedAt': datetime.datetime(2025, 5, 19, 9, 21, 17, 215727, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2025, 5, 19, 9, 21, 17, 215727, tzinfo=tzlocal())}
{ 'dataSourceId': 'QBWS4OTNVT',
  'ingestionJobId': 'P6KNUIOXZ8',
  'knowledgeBaseId': 'UVJRQDUOEV',
  'startedAt': date

## Creating HR Agent

Create the HR agent that will have an `Amazon Bedrock Knowledge Base` with comprehensive HR and travel policies as well as `action groups` to handle employee information retrieval and travel policy enforcement. This agent will serve as the central knowledge and identity provider for the travel booking system.

In order to have an effective HR agent, it is important to set clear instructions of what the agent should do and what it should not do. It is also important to provide clear definitions for when the agent should use the knowledge bases and action groups available to it.

We will provide the following instructions to our HR agent:
You are an HR Assistant that helps employees navigate company travel policies and processes travel-related requests.

Your capabilities include:

Retrieving employee information and travel preferences

Providing guidance on company travel policies

Validating travel requests against policy rules

Determining approval requirements for travel requests

Checking passport and visa requirements

Core behaviors:

Always verify employee identity before providing personalized information

Maintain strict confidentiality of employee data

Provide clear explanations of policy rules and requirements

Enforce travel policies consistently across all employee grades

Escalate exceptions to appropriate approval levels

Present information in a professional, helpful manner

DO NOT approve travel requests that violate company policy

DO NOT share one employee's information with another employee

Response style:

Be helpful and solution-oriented

Use clear, concise language

Cite specific policy sections when explaining restrictions

Maintain a professional yet friendly tone

Provide actionable next steps

Be thorough but avoid unnecessary details


We will also connect a knowledge base for HR and travel policies with the following instructions:

Access this knowledge base when needing to explain or enforce specific travel policies, approval workflows, or expense reimbursement guidelines.

And we will make the following tools available to the agent:
- `get_employee_info`: Retrieves employee details including grade, department, and manager
- `get_travel_preferences`: Retrieves employee's travel preferences and requirements
- `validate_travel_request`: Checks if a travel request complies with company policy
- `get_approval_requirements`: Determines approval workflow based on destination, duration, and cost
- `check_passport_status`: Verifies if employee's passport is valid for international travel

In [17]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [18]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': """Access this knowledge base when needing to explain work travel relate policy ."""
}

In [19]:
agent_description = """You are an HR Assistant that helps employees navigate company travel policies and processes travel-related requests."""

agent_instruction = """You are an HR Assistant that helps employees navigate company travel policies, processes travel-related requests, and manages the approval workflow for travel bookings.

Your capabilities include:
1. Retrieving employee information and travel preferences
2. Providing guidance on company travel policies
3. Validating travel requests against policy rules
4. Determining approval requirements for travel requests
5. Checking passport and visa requirements
6. Creating and managing travel approval requests
7. Tracking approval status for pending requests
8. Processing approvals from authorized approvers

Core behaviors:
1. Always verify employee identity before providing personalized information .i.e ask for employee id before providing any information
2. Maintain a professional yet conversational tone
3. Provide clear explanations of policy rules and requirements
4. Present information in an easy-to-understand manner
5. Enforce travel policies consistently across all employee grades
6. Escalate exceptions to appropriate approval levels
7. Present information in a professional, helpful manner
8. DO NOT approve travel requests that violate company policy
9. DO NOT share one employee's information with another employee
10. ALWAYS create an approval request before allowing any travel booking to proceed
11. Verify approval status before confirming any travel arrangements
12. Only accept approvals from authorized approvers (self, manager, director, or VP as required)
13. Clearly communicate the approval status and next steps to employees

Approval Workflow:
1. When an employee initiates a travel request, validate it against company policy
2. Determine the required approval level based on destination, duration, cost, and employee grade
3. Create an approval request with all relevant details
4. Inform the employee about the approval requirements and provide the request ID
5. When asked about approval status, check and communicate the current status
6. When an approver attempts to approve a request, verify their authority before processing
7. Checking visa requirements for international travel
8. Generating visa application documents
9. Once approved, allow the booking process to proceed
10. If a request is pending approval, remind the employee to obtain the necessary approvals

We will also connect a knowledge base for HR and travel policies with the following instructions:

Access this knowledge base when needing to explain or enforce specific travel policies, approval workflows, or expense reimbursement guidelines.

When handling visa-related queries:
1. Check if the employee needs a visa based on their nationality and destination
2. Verify if they already have a valid visa for the destination
3. Provide information about required documents and processing times
4. Generate visa application documents when requested
5. Remind users about visa processing lead times when planning international travel

And we will make the following tools available to the agent:
- `get_employee_info`: Retrieves employee details including grade, department, and manager
- `get_travel_preferences`: Retrieves employee's travel preferences and requirements
- `validate_travel_request`: Checks if a travel request complies with company policy
- `get_approval_requirements`: Determines approval workflow based on destination, duration, and cost
- `check_passport_status`: Verifies if employee's passport is valid for international travel
- `create_approval_request`: Creates a new travel approval request in the system
- `check_approval_status`: Checks the status of an existing approval request
- `approve_request`: Processes an approval from an authorized approver
- `list_pending_approvals`: Lists all pending approvals for a manager
- `check_visa_requirements`: Checks if an employee needs a visa for a specific destination
- `generate_visa_application_documents`: Generates visa application documents based on employee information

Response style:
- Be helpful and solution-oriented
- Use clear, concise language
- Cite specific policy sections when explaining restrictions
- Maintain natural conversation flow
- Maintain a professional yet friendly tone
- Provide actionable next steps
- Be thorough but avoid unnecessary details
- do not add extra information not required by the user
"""

hr_agent = agents.create_agent(
    hr_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn],
    code_interpretation=True
)

hr_agent

Waiting for agent status to change. Current status CREATING
Agent id XQJ6KAV7P1 current status: NOT_PREPARED


('XQJ6KAV7P1',
 'TSTALIASID',
 'arn:aws:bedrock:us-west-2:225584133776:agent-alias/XQJ6KAV7P1/TSTALIASID')

### Associating knowledge base
Now that we've created the agent, let's associate the previously created knowledge base to it.

In [21]:
agents.associate_kb_with_agent(
    hr_agent[0],
    kb_config['kb_instruction'],
    kb_config['kb_id']
)

### Creating Lambda

In order to enable the agent to execute tasks, we will create an AWS Lambda function that implements the tasks execution. We will then provide this lambda function to the agent action group.
On this block, we're going to generate Lambda function Code:

In [53]:
%%writefile hr_agent_lambda.py
import boto3
import json
import os
import uuid
from datetime import datetime, timedelta
from boto3.dynamodb.conditions import Key, Attr

# Initialize DynamoDB resources
dynamodb_resource = boto3.resource('dynamodb')
dynamodb_table = os.getenv('dynamodb_table')
dynamodb_pk = os.getenv('dynamodb_pk')
dynamodb_sk = os.getenv('dynamodb_sk')

approval_requests_table = os.getenv('approval_requests_table')
approval_pk = os.getenv('approval_pk')
approval_sk = os.getenv('emp_id')
# Helper functions
def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def read_dynamodb(table_name, pk_field, pk_value, filter_key=None, filter_value=None):
    try:
        table = dynamodb_resource.Table(table_name)
        key_expression = Key(pk_field).eq(pk_value)
        
        if filter_key:
            filter_expression = Attr(filter_key).eq(filter_value)
            query_data = table.query(
                KeyConditionExpression=key_expression,
                FilterExpression=filter_expression
            )
        else:
            query_data = table.query(
                KeyConditionExpression=key_expression
            )
        
        return query_data['Items']
    except Exception as e:
        print(f'Error querying table: {table_name}. Error: {str(e)}')
        return []

def update_dynamodb(table_name, pk_field, pk_value, update_field, update_value):
    try:
        table = dynamodb_resource.Table(table_name)
        response = table.update_item(
            Key={pk_field: pk_value},
            UpdateExpression=f"set {update_field} = :val",
            ExpressionAttributeValues={':val': update_value},
            ReturnValues="UPDATED_NEW"
        )
        return response
    except Exception as e:
        print(f'Error updating table: {table_name}. Error: {str(e)}')
        return None

# Core HR functions
def get_employee_info(emp_id):
    """Retrieves employee details including grade, department, and manager"""
    employee_data = read_dynamodb(dynamodb_table, dynamodb_pk, emp_id)
    
    if not employee_data:
        return f"No employee found with ID: {emp_id}"
    
    # Remove sensitive fields before returning
    employee = employee_data[0]
    if 'emergency_contact' in employee:
        del employee['emergency_contact']
    
    return employee

def get_travel_preferences(emp_id):
    """Retrieves employee's travel preferences and requirements"""
    employee_data = read_dynamodb(dynamodb_table, dynamodb_pk, emp_id)
    
    if not employee_data:
        return f"No employee found with ID: {emp_id}"
    
    # Extract only travel-related preferences
    travel_fields = ['preferred_airlines', 'dietary_restrictions', 'accessibility_needs']
    travel_preferences = {k: employee_data[0].get(k, 'Not specified') for k in travel_fields}
    
    return travel_preferences

def validate_travel_request(emp_id, destination, duration, cost):
    """Checks if a travel request complies with company policy"""
    employee_data = read_dynamodb(dynamodb_table, dynamodb_pk, emp_id)
    
    if not employee_data:
        return f"No employee found with ID: {emp_id}"
    
    employee = employee_data[0]
    grade = employee.get('grade', '')
    budget_remaining = float(employee.get('travel_budget_remaining', 0))
    
    # Policy validation logic
    validation_results = {
        "valid": True,
        "issues": [],
        "budget_sufficient": budget_remaining >= float(cost)
    }
    
    # Check budget
    if float(cost) > budget_remaining:
        validation_results["valid"] = False
        validation_results["issues"].append(f"Insufficient budget: {budget_remaining} remaining, {cost} requested")
    
    # Check duration limits by grade
    max_duration = {"Junior": 5, "Mid-level": 7, "Senior": 10, "Executive": 14}
    if int(duration) > max_duration.get(grade, 5):
        validation_results["valid"] = False
        validation_results["issues"].append(f"Duration exceeds limit for {grade} grade: {duration} days requested, {max_duration.get(grade, 5)} allowed")
    
    # Check high-risk destinations (simplified example)
    high_risk_destinations = ["Country A", "Country B", "Country C"]
    if destination in high_risk_destinations:
        validation_results["valid"] = False
        validation_results["issues"].append(f"Destination {destination} requires special approval")
    
    return validation_results

def get_approval_requirements(emp_id, destination, duration, cost):
    """Determines approval workflow based on destination, duration, and cost"""
    employee_data = read_dynamodb(dynamodb_table, dynamodb_pk, emp_id)
    
    if not employee_data:
        return f"No employee found with ID: {emp_id}"
    
    employee = employee_data[0]
    grade = employee.get('grade', '')
    default_approval = employee.get('approval_level', 'Manager')
    
    # Determine approval level based on various factors
    approval_level = default_approval
    
    # Cost thresholds by grade
    cost_thresholds = {
        "Junior": 1000,
        "Mid-level": 2000,
        "Senior": 5000,
        "Executive": 10000
    }
    
    # Duration thresholds
    if int(duration) > 7:
        approval_level = "Director"
    
    # Cost thresholds
    if float(cost) > cost_thresholds.get(grade, 1000):
        approval_level = "Director"
    
    if float(cost) > 10000:
        approval_level = "VP"
    
    # International travel always requires higher approval
    international_destinations = ["Country X", "Country Y", "Country Z"]  # Example list
    if destination in international_destinations:
        if approval_level == "Manager":
            approval_level = "Director"
    
    return {
        "approval_required": approval_level,
        "manager_id": employee.get('manager_id', 'Unknown'),
        "estimated_approval_time": "24-48 hours" if approval_level == "Manager" else "3-5 business days"
    }
def check_visa_requirements(emp_id, destination_country):
    """Checks if an employee needs a visa for a specific destination"""
    try:
        # Get employee details
        table = dynamodb_resource.Table(dynamodb_table)
        response = table.get_item(
            Key={'emp_id': emp_id}
        )
        
        if 'Item' not in response:
            return {"status": "Error", "message": "Employee not found"}
        
        employee = response['Item']
        nationality = employee.get('nationality', 'Unknown')
        
        # Visa requirement database (simplified)
        # In a real system, this would be a comprehensive database or API call
        visa_requirements = {
            "United States": ["India", "China", "Brazil", "Russia"],
            "United Kingdom": ["India", "China", "Russia", "Brazil"],
            "Schengen": ["India", "China", "Russia", "Brazil"],
            "Japan": ["India", "China", "Russia"],
            "Australia": ["India", "China", "Russia", "Brazil"]
        }
        
        # Check if employee already has a valid visa
        employee_visas = employee.get('visas', {})
        if destination_country in employee_visas:
            visa_status = employee_visas[destination_country]
            if visa_status.get('status') == 'Valid' and datetime.strptime(visa_status.get('expiry', '2000-01-01'), '%Y-%m-%d') > datetime.now():
                return {
                    "status": "Success",
                    "needs_visa": False,
                    "has_valid_visa": True,
                    "visa_expiry": visa_status.get('expiry')
                }
        
        # Check if visa is required
        needs_visa = False
        if destination_country in visa_requirements:
            if nationality in visa_requirements[destination_country]:
                needs_visa = True
        
        # Check passport validity (required for visa application)
        passport_status = employee.get('passport_status', 'Unknown')
        passport_expiry = employee.get('passport_expiry', 'Unknown')
        
        # Calculate processing time (simplified)
        processing_times = {
            "United States": 30,
            "United Kingdom": 15,
            "Schengen": 15,
            "Japan": 7,
            "Australia": 20
        }
        
        processing_days = processing_times.get(destination_country, 14)
        
        return {
            "status": "Success",
            "needs_visa": needs_visa,
            "has_valid_visa": False,
            "nationality": nationality,
            "destination": destination_country,
            "passport_status": passport_status,
            "passport_expiry": passport_expiry,
            "estimated_processing_days": processing_days,
            "required_documents": ["Passport", "Invitation Letter", "Travel Itinerary", "Hotel Booking", "Bank Statement"]
        }
    except Exception as e:
        return {"status": "Error", "message": str(e)}

def generate_visa_application_documents(emp_id, destination_country, travel_purpose, travel_dates):
    """Generates visa application documents based on employee information"""
    try:
        # Get employee details
        table = dynamodb_resource.Table(dynamodb_table)
        response = table.get_item(
            Key={'emp_id': emp_id}
        )
        
        if 'Item' not in response:
            return {"status": "Error", "message": "Employee not found"}
        
        employee = response['Item']
        
        # In a real implementation, this would generate actual documents
        # For this example, we'll just return document information
        
        document_url = f"https://s3.amazonaws.com/visa-applications/{emp_id}_{destination_country}.pdf"
        
        return {
            "status": "Success",
            "document_url": document_url,
            "application_id": str(uuid.uuid4()),
            "required_documents": [
                "Passport (valid for at least 6 months beyond stay)",
                "Visa Application Form (completed)",
                "Travel Itinerary",
                "Hotel Reservation",
                "Employment Verification Letter",
                "Bank Statements (last 3 months)"
            ],
            "submission_instructions": f"Submit all documents to the {destination_country} embassy or consulate",
            "processing_time": "15-30 business days"
        }
    except Exception as e:
        return {"status": "Error", "message": str(e)}

def check_passport_status(emp_id):
    """Verifies if employee's passport is valid for international travel"""
    employee_data = read_dynamodb(dynamodb_table, dynamodb_pk, emp_id)
    
    if not employee_data:
        return f"No employee found with ID: {emp_id}"
    
    employee = employee_data[0]
    passport_status = employee.get('passport_status', 'Unknown')
    passport_expiry = employee.get('passport_expiry', 'Unknown')
    
    # Check if passport is expiring soon (within 6 months)
    if passport_expiry != 'Unknown':
        try:
            expiry_date = datetime.strptime(passport_expiry, '%Y-%m-%d')
            six_months_from_now = datetime.now() + timedelta(days=180)
            
            if expiry_date < datetime.now():
                passport_status = "Expired"
            elif expiry_date < six_months_from_now:
                passport_status = "Expiring Soon"
        except:
            pass
    
    return {
        "passport_status": passport_status,
        "passport_expiry": passport_expiry,
        "valid_for_international_travel": passport_status == "Valid",
        "nationality": employee.get('nationality', 'Unknown')
    }


def create_approval_request(emp_id, manager_id, request_type, details, approval_level):
    """Creates a new approval request in the system"""
    request_id = str(uuid.uuid4())
    timestamp = datetime.now().isoformat()
    
    item = {
        'request_id': request_id,
        'emp_id': emp_id,
        'manager_id': manager_id,
        'request_type': request_type,  # e.g., "hotel", "flight", "car"
        'details': details,            # JSON string with booking details
        'approval_level': approval_level,  # "Self", "Manager", "Director", "VP"
        'status': 'Pending',
        'created_at': timestamp,
        'updated_at': timestamp
    }
    
    table = dynamodb_resource.Table(approval_requests_table)
    table.put_item(Item=item)
    
    return {
        "request_id": request_id,
        "status": "Pending",
        "message": f"Approval request created and pending {approval_level} approval"
    }

def check_approval_status(request_id, emp_id):
    """Checks the status of an approval request"""
    table = dynamodb_resource.Table(approval_requests_table)
    response = table.get_item(
        Key={
            'request_id': request_id,
            'emp_id': emp_id
        }
    )
    
    if 'Item' not in response:
        return {"status": "Not Found", "message": "Approval request not found"}
    
    return {
        "status": response['Item']['status'],
        "approval_level": response['Item']['approval_level'],
        "details": response['Item']['details'],
        "created_at": response['Item']['created_at'],
        "updated_at": response['Item']['updated_at']
    }

def approve_request(request_id, emp_id, approver_id):
    """Approves a pending request"""
    # First check if the approver is authorized
    table = dynamodb_resource.Table(approval_requests_table)
    response = table.get_item(
        Key={
            'request_id': request_id,
            'emp_id': emp_id
        }
    )
    
    if 'Item' not in response:
        return {"status": "Error", "message": "Request not found"}
    
    request = response['Item']
    
    # For self-approval, the approver must be the employee
    if request['approval_level'] == 'Self' and approver_id != emp_id:
        return {"status": "Error", "message": "Unauthorized approval attempt"}
    
    # For manager approval, check if approver is the manager
    if request['approval_level'] != 'Self' and approver_id != request['manager_id']:
        # Here you would check if the approver is a Director or VP if needed
        # This would require looking up the approver's role in the employee table
        employee_table = dynamodb_resource.Table(dynamodb_table)
        approver_data = employee_table.get_item(
            Key={'emp_id': approver_id}
        )
        
        if 'Item' not in approver_data or approver_data['Item']['grade'] not in ['Director', 'Executive']:
            return {"status": "Error", "message": "Unauthorized approval attempt"}
    
    # Update the request status
    timestamp = datetime.now().isoformat()
    table.update_item(
        Key={
            'request_id': request_id,
            'emp_id': emp_id
        },
        UpdateExpression="set #status = :s, approver_id = :a, updated_at = :u",
        ExpressionAttributeNames={
            '#status': 'status'
        },
        ExpressionAttributeValues={
            ':s': 'Approved',
            ':a': approver_id,
            ':u': timestamp
        }
    )
    
    return {
        "status": "Approved",
        "message": f"Request {request_id} has been approved by {approver_id}",
        "updated_at": timestamp
    }

def list_pending_approvals(approver_id):
    """Lists all pending approvals for a manager"""
    table = dynamodb_resource.Table(approval_requests_table)
    
    # Query for requests where this person is the manager and status is Pending
    response = table.scan(
        FilterExpression="manager_id = :m AND #status = :s",
        ExpressionAttributeNames={
            '#status': 'status'
        },
        ExpressionAttributeValues={
            ':m': approver_id,
            ':s': 'Pending'
        }
    )
    
    return response['Items']

def lambda_handler(event, context):
    print(event)
    
    # Name of the function to invoke
    function = event.get('function', '')
    
    # Parameters to invoke function with
    parameters = event.get('parameters', [])
    
    # Route to appropriate function
    if function == 'get_employee_info':
        emp_id = get_named_parameter(event, "emp_id")
        result = get_employee_info(emp_id)
    elif function == 'get_travel_preferences':
        emp_id = get_named_parameter(event, "emp_id")
        result = get_travel_preferences(emp_id)
    elif function == 'validate_travel_request':
        emp_id = get_named_parameter(event, "emp_id")
        destination = get_named_parameter(event, "destination")
        duration = get_named_parameter(event, "duration")
        cost = get_named_parameter(event, "cost")
        result = validate_travel_request(emp_id, destination, duration, cost)
    elif function == 'get_approval_requirements':
        emp_id = get_named_parameter(event, "emp_id")
        destination = get_named_parameter(event, "destination")
        duration = get_named_parameter(event, "duration")
        cost = get_named_parameter(event, "cost")
        result = get_approval_requirements(emp_id, destination, duration, cost)
    elif function == 'check_passport_status':
        emp_id = get_named_parameter(event, "emp_id")
        result = check_passport_status(emp_id)
    # New approval workflow functions
    elif function == 'create_approval_request':
        emp_id = get_named_parameter(event, "emp_id")
        request_type = get_named_parameter(event, "request_type")
        details = get_named_parameter(event, "details")
        
        # Get employee info to determine manager and approval level
        employee_info = get_employee_info(emp_id)
        manager_id = employee_info.get('manager_id', 'None')
        approval_level = employee_info.get('approval_level', 'Manager')
        
        result = create_approval_request(emp_id, manager_id, request_type, details, approval_level)
    elif function == 'check_approval_status':
        request_id = get_named_parameter(event, "request_id")
        emp_id = get_named_parameter(event, "emp_id")
        result = check_approval_status(request_id, emp_id)
    elif function == 'approve_request':
        request_id = get_named_parameter(event, "request_id")
        emp_id = get_named_parameter(event, "emp_id")
        approver_id = get_named_parameter(event, "approver_id")
        result = approve_request(request_id, emp_id, approver_id)
    elif function == 'list_pending_approvals':
        approver_id = get_named_parameter(event, "approver_id")
        result = list_pending_approvals(approver_id)
    else:
        result = f"Error: Function '{function}' not recognized"

    # Format and return the response
    response = populate_function_response(event, result)
    print(response)
    return response


Overwriting hr_agent_lambda.py


### Defining available actions

Next we will define the available actions that an agent can perform using [Function Details]. 

In [54]:
hr_functions_def = [
    {
        "name": "get_employee_info",
        "description": """Retrieves employee details including grade, department, and manager""",
        "parameters": {
            "emp_id": {
                "description": "Unique employee identifier",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "get_travel_preferences",
        "description": """Retrieves employee's travel preferences and requirements""",
        "parameters": {
            "emp_id": {
                "description": "Unique employee identifier",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "validate_travel_request",
        "description": """Checks if a travel request complies with company policy""",
        "parameters": {
            "emp_id": {
                "description": "Unique employee identifier",
                "required": True,
                "type": "string"
            },
            "destination": {
                "description": "Travel destination (city or country)",
                "required": True,
                "type": "string"
            },
            "duration": {
                "description": "Trip duration in days",
                "required": True,
                "type": "integer"
            },
            "cost": {
                "description": "Estimated total cost of the trip",
                "required": True,
                "type": "number"
            }
        }
    },
    {
        "name": "get_approval_requirements",
        "description": """Determines approval workflow based on destination, duration, and cost""",
        "parameters": {
            "emp_id": {
                "description": "Unique employee identifier",
                "required": True,
                "type": "string"
            },
            "destination": {
                "description": "Travel destination (city or country)",
                "required": True,
                "type": "string"
            },
            "duration": {
                "description": "Trip duration in days",
                "required": True,
                "type": "integer"
            },
            "cost": {
                "description": "Estimated total cost of the trip",
                "required": True,
                "type": "number"
            }
        }
    },
    {
        "name": "check_passport_status",
        "description": """Verifies if employee's passport is valid for international travel""",
        "parameters": {
            "emp_id": {
                "description": "Unique employee identifier",
                "required": True,
                "type": "string"
            }
        }
    },{
        "name": "create_approval_request",
        "description": """Creates a new travel approval request in the system""",
        "parameters": {
            "emp_id": {
                "description": "Employee ID requesting approval",
                "required": True,
                "type": "string"
            },
            "request_type": {
                "description": "Type of travel request (hotel, flight, car)",
                "required": True,
                "type": "string"
            },
            "details": {
                "description": "JSON string with booking details",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "check_approval_status",
        "description": """Checks the status of an approval request""",
        "parameters": {
            "request_id": {
                "description": "Unique request identifier",
                "required": True,
                "type": "string"
            },
            "emp_id": {
                "description": "Employee ID who made the request",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "approve_request",
        "description": """Approves a pending travel request""",
        "parameters": {
            "request_id": {
                "description": "Unique request identifier",
                "required": True,
                "type": "string"
            },
            "emp_id": {
                "description": "Employee ID who made the request",
                "required": True,
                "type": "string"
            },
            "approver_id": {
                "description": "Employee ID of the approver",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "list_pending_approvals",
        "description": """Lists all pending approvals for a manager""",
        "parameters": {
            "approver_id": {
                "description": "Employee ID of the approver/manager",
                "required": True,
                "type": "string"
            }
        }
    },{
    "name": "check_visa_requirements",
    "description": """Checks if an employee needs a visa for a specific destination""",
    "parameters": {
        "emp_id": {
            "description": "Employee ID",
            "required": True,
            "type": "string"
        },
        "destination_country": {
            "description": "Destination country for travel",
            "required": True,
            "type": "string"
        }
    }
},
{
    "name": "generate_visa_application_documents",
    "description": """Generates visa application documents based on employee information""",
    "parameters": {
        "emp_id": {
            "description": "Employee ID",
            "required": True,
            "type": "string"
        },
        "destination_country": {
            "description": "Destination country for travel",
            "required": True,
            "type": "string"
        },
        "travel_purpose": {
            "description": "Purpose of travel (business, conference, etc.)",
            "required": True,
            "type": "string"
        },
        "travel_dates": {
            "description": "Travel dates in format 'YYYY-MM-DD to YYYY-MM-DD'",
            "required": True,
            "type": "string"
        }
    }
}
]


### Creating action group and attaching to the agent
Now it's time to add this Lambda function and the function details as an action group for this agent and prepare it.

In [56]:
agents.add_action_group_with_lambda(
    agent_name=hr_agent_name,
    lambda_function_name=hr_lambda_name,
    source_code_file="hr_agent_lambda.py",
    agent_functions=hr_functions_def,
    agent_action_group_name="hr_policy_actions",
    agent_action_group_description="Functions to retrieve employee information and enforce travel policies",
    dynamo_args=dynamoDB_args
)



Table hr-agent-348d2ff0-users already exists, skipping table creation step


## Loading data to DynamoDB

Now that we've created our agent, let's load some generated data to DynamoDB. That will allow the agent to interact with some live data to perform actions

In [52]:
agents.create_dynamodb(
    approval_requests_table,
    approval_pk,
    approval_sk
)



In [57]:
# Create sample employee data with visa information
employee_data = [
    {
        "emp_id": "E001",
        "name": "John Smith",
        "email": "john.smith@company.com",
        "grade": "Senior",
        "department": "Engineering",
        "manager_id": "E005",
        "nationality": "United States",
        "passport_status": "Valid",
        "passport_expiry": "2026-05-15",
        "preferred_airlines": ["United", "Delta", "American"],
        "dietary_restrictions": "None",
        "accessibility_needs": "None",
        "emergency_contact": {"name": "Jane Smith", "phone": "555-123-4567"},
        "travel_budget_remaining": 5000,
        "approval_level": "Manager",
        "visas": {
            "United Kingdom": {
                "status": "Valid",
                "type": "Business",
                "expiry": "2025-06-20",
                "multiple_entry": True
            },
            "Schengen": {
                "status": "Valid",
                "type": "Business",
                "expiry": "2024-12-15",
                "multiple_entry": True
            },
            "Japan": {
                "status": "Expired",
                "type": "Business",
                "expiry": "2023-10-10",
                "multiple_entry": False
            }
        },
        "travel_documents": {
            "passport_number": "US123456789",
            "national_id": "123-45-6789",
            "has_global_entry": True
        }
    },
    {
        "emp_id": "E002",
        "name": "Maria Garcia",
        "email": "maria.garcia@company.com",
        "grade": "Mid-level",
        "department": "Marketing",
        "manager_id": "E006",
        "nationality": "Spain",
        "passport_status": "Valid",
        "passport_expiry": "2025-08-22",
        "preferred_airlines": ["Iberia", "British Airways"],
        "dietary_restrictions": "Vegetarian",
        "accessibility_needs": "None",
        "emergency_contact": {"name": "Carlos Garcia", "phone": "555-234-5678"},
        "travel_budget_remaining": 3500,
        "approval_level": "Manager",
        "visas": {
            "United States": {
                "status": "Valid",
                "type": "Business",
                "expiry": "2025-03-15",
                "multiple_entry": True
            },
            "United Kingdom": {
                "status": "Valid",
                "type": "Business",
                "expiry": "2024-11-30",
                "multiple_entry": True
            }
        },
        "travel_documents": {
            "passport_number": "ESP987654321",
            "national_id": "X1234567Z",
            "has_global_entry": False
        }
    },
    {
        "emp_id": "E003",
        "name": "Raj Patel",
        "email": "raj.patel@company.com",
        "grade": "Junior",
        "department": "Finance",
        "manager_id": "E007",
        "nationality": "India",
        "passport_status": "Expiring",
        "passport_expiry": "2024-12-10",
        "preferred_airlines": ["Emirates", "Air India"],
        "dietary_restrictions": "Vegetarian",
        "accessibility_needs": "None",
        "emergency_contact": {"name": "Priya Patel", "phone": "555-345-6789"},
        "travel_budget_remaining": 2000,
        "approval_level": "Director",
        "visas": {
            "United States": {
                "status": "Pending",
                "type": "Business",
                "application_date": "2024-06-01",
                "estimated_processing_time": "30 days"
            },
            "United Kingdom": {
                "status": "Valid",
                "type": "Business",
                "expiry": "2025-01-15",
                "multiple_entry": False
            },
            "Schengen": {
                "status": "Valid",
                "type": "Business",
                "expiry": "2024-09-30",
                "multiple_entry": True
            }
        },
        "travel_documents": {
            "passport_number": "IND543216789",
            "national_id": "ABCDE1234F",
            "has_global_entry": False
        }
    },
    {
        "emp_id": "E004",
        "name": "Sarah Johnson",
        "email": "sarah.johnson@company.com",
        "grade": "Executive",
        "department": "Executive",
        "manager_id": "None",
        "nationality": "Canada",
        "passport_status": "Valid",
        "passport_expiry": "2028-03-30",
        "preferred_airlines": ["Air Canada", "United"],
        "dietary_restrictions": "Gluten-free",
        "accessibility_needs": "None",
        "emergency_contact": {"name": "Michael Johnson", "phone": "555-456-7890"},
        "travel_budget_remaining": 15000,
        "approval_level": "Self",
        "visas": {
            "United States": {
                "status": "Valid",
                "type": "Business",
                "expiry": "2027-05-20",
                "multiple_entry": True
            },
            "United Kingdom": {
                "status": "Valid",
                "type": "Business",
                "expiry": "2026-08-15",
                "multiple_entry": True
            },
            "Japan": {
                "status": "Valid",
                "type": "Business",
                "expiry": "2025-11-10",
                "multiple_entry": True
            },
            "Australia": {
                "status": "Valid",
                "type": "Business",
                "expiry": "2026-02-28",
                "multiple_entry": True
            }
        },
        "travel_documents": {
            "passport_number": "CA98765432",
            "national_id": "123-456-789",
            "has_global_entry": True,
            "has_nexus": True
        }
    },
    {
        "emp_id": "E005",
        "name": "David Lee",
        "email": "david.lee@company.com",
        "grade": "Executive",
        "department": "Engineering",
        "manager_id": "E004",
        "nationality": "South Korea",
        "passport_status": "Valid",
        "passport_expiry": "2027-07-18",
        "preferred_airlines": ["Korean Air", "Delta"],
        "dietary_restrictions": "None",
        "accessibility_needs": "None",
        "emergency_contact": {"name": "Michelle Lee", "phone": "555-567-8901"},
        "travel_budget_remaining": 12000,
        "approval_level": "VP",
        "visas": {
            "United States": {
                "status": "Valid",
                "type": "Business",
                "expiry": "2026-09-15",
                "multiple_entry": True
            },
            "United Kingdom": {
                "status": "Valid",
                "type": "Business",
                "expiry": "2025-12-20",
                "multiple_entry": True
            },
            "Schengen": {
                "status": "Valid",
                "type": "Business",
                "expiry": "2026-03-10",
                "multiple_entry": True
            }
        },
        "travel_documents": {
            "passport_number": "KR87654321",
            "national_id": "123456-7890123",
            "has_global_entry": True
        }
    }
]

# Write the data to a file
with open("hr_employee_data.json", "w") as f:
    for employee in employee_data:
        f.write(json.dumps(employee) + "\n")

# Load the data into DynamoDB
with open("hr_employee_data.json") as f:
    table_items = [json.loads(line) for line in f]
    
agents.load_dynamodb(dynamodb_table, table_items)


Testing that data was loaded on DynamoDB

# Query for employee with ID E005
employee_query = agents.query_dynamodb(
    dynamodb_table,  # The HR agent's DynamoDB table name
    dynamodb_pk,     # Primary key field name ("emp_id")
    "E005",          # Employee ID to query for
    None,            # No sort key for the HR table
    None             # No sort key value needed
)

# Display the query results
employee_query


## Testing HR Agent

Now, let's run some tests on the agent we just created to make sure it's working. To do so we will use our test alias: `TSTALIASID` which allows you to invoke a draft version of your agent

In [43]:
%%time
response = agents.invoke(
    """Can you tell me about employee E005? 
    What are their travel preferences and what approval level do they need for travel requests?""", 
    hr_agent[0], enable_trace=True
)
print("====================")
print(response)

invokeAgent API request ID: 70877956-eaff-44eb-9f0d-4a97b4a6a98d
invokeAgent API session ID: 572fb338-348f-11f0-b503-06efa4e78d78
---- Step 1 ----
Took 4.9s, using 2479 tokens (in: 2267, out: 212) to complete prior action, observe, orchestrate.
To provide you with accurate information, I'll need to use a few functions to retrieve the necessary details. First, I'll get the employee information, then their travel preferences, and finally, I'll check the approval requirements for a hypothetical trip to determine their approval level.
Using tool: get_employee_info with these inputs:
[{'name': 'emp_id', 'type': 'string', 'value': 'E005'}]

--tool outputs:
{'nationality': 'South Korea', 'department': 'Engineering', 'grade': 'Executive', 'dietary_restrictions': 'None', 'emp_id': 'E005', 'email': 'david.lee@company.com', 'travel_budget_remaining': Decimal('12000'), 'name': 'David Lee', 'approval_level': 'VP', 'manager_id': 'E004', 'accessibility_needs':...

Using tool: get_travel_preferences w

In [45]:
%%time
response = agents.invoke(
    """I'm employee E003 and I want to travel to Spain for 10 days. 
    The trip will cost $4000. Is this request compliant with company policy?""", 
    hr_agent[0], enable_trace=True
)
print("====================")
print(response)


invokeAgent API request ID: 7603909e-1960-431c-9949-21ac55dca7bc
invokeAgent API session ID: 572fb338-348f-11f0-b503-06efa4e78d78
---- Step 1 ----
Took 2.8s, using 3269 tokens (in: 3163, out: 106) to complete prior action, observe, orchestrate.
Before I can provide information about the travel request compliance, I need to verify the employee's identity and retrieve their information. Then, I'll validate the travel request against company policy.
Using tool: get_employee_info with these inputs:
[{'name': 'emp_id', 'type': 'string', 'value': 'E003'}]

--tool outputs:
{'nationality': 'India', 'department': 'Finance', 'grade': 'Junior', 'dietary_restrictions': 'Vegetarian', 'emp_id': 'E003', 'email': 'raj.patel@company.com', 'travel_budget_remaining': Decimal('2000'), 'name': 'Raj Patel', 'approval_level': 'Director', 'manager_id': 'E007', 'accessibility_needs': '...

---- Step 2 ----
Took 3.7s, using 3586 tokens (in: 3439, out: 147) to complete prior action, observe, orchestrate.
Now tha

In [47]:
%%time
response = agents.invoke(
    """I'm employee E003 and I need to travel internationally next month.
    Is my passport valid for international travel?""", 
    hr_agent[0], enable_trace=True
)
print("====================")
print(response)

invokeAgent API request ID: 1f7fc298-cb1f-452a-b4a2-404e2fcb3756
invokeAgent API session ID: 572fb338-348f-11f0-b503-06efa4e78d78
---- Step 1 ----
Took 2.9s, using 4085 tokens (in: 3979, out: 106) to complete prior action, observe, orchestrate.
To answer this question, I need to verify the employee's identity and check their passport status. I'll use the employee information function first, and then the passport status check function.
Using tool: get_employee_info with these inputs:
[{'name': 'emp_id', 'type': 'string', 'value': 'E003'}]

--tool outputs:
{'nationality': 'India', 'department': 'Finance', 'grade': 'Junior', 'dietary_restrictions': 'Vegetarian', 'emp_id': 'E003', 'email': 'raj.patel@company.com', 'travel_budget_remaining': Decimal('2000'), 'name': 'Raj Patel', 'approval_level': 'Director', 'manager_id': 'E007', 'accessibility_needs': '...

---- Step 2 ----
Took 2.9s, using 4347 tokens (in: 4255, out: 92) to complete prior action, observe, orchestrate.
I've verified the em

## Create alias for HR Agent

Since we've tested and validated our HR agent, let's now create an alias for it
This alias will allow the HR agent to be used in multi-agent collaboration


In [48]:
hr_agent_alias_id, hr_agent_alias_arn = agents.create_agent_alias(
    hr_agent[0], 'v1'
)

print(f"HR Agent Alias ID: {hr_agent_alias_id}")
print(f"HR Agent Alias ARN: {hr_agent_alias_arn}")

HR Agent Alias ID: JUZ0EVGCPO
HR Agent Alias ARN: arn:aws:bedrock:us-west-2:225584133776:agent-alias/XQJ6KAV7P1/JUZ0EVGCPO


## Saving HR Agent Information
# Store environment variables to be used in other notebooks




In [49]:
# Get the HR agent ARN
hr_agent_arn = agents.get_agent_arn_by_name(hr_agent_name)
hr_agent_id = hr_agent[0]  # Assuming hr_agent_id was set when creating the agent
hr_kb = knowledge_base_name
hr_dynamodb = dynamodb_table

# Store variables for use in other notebooks
%store hr_agent_arn
%store hr_agent_alias_arn
%store hr_agent_alias_id
%store hr_lambda_name
%store hr_agent_name
%store hr_agent_id
%store hr_kb
%store hr_dynamodb

Stored 'hr_agent_arn' (str)
Stored 'hr_agent_alias_arn' (str)
Stored 'hr_agent_alias_id' (str)
Stored 'hr_lambda_name' (str)
Stored 'hr_agent_name' (str)
Stored 'hr_agent_id' (str)
Stored 'hr_kb' (str)
Stored 'hr_dynamodb' (str)


In [50]:
hr_agent_arn, hr_agent_alias_arn, hr_agent_alias_id

('arn:aws:bedrock:us-west-2:225584133776:agent/XQJ6KAV7P1',
 'arn:aws:bedrock:us-west-2:225584133776:agent-alias/XQJ6KAV7P1/JUZ0EVGCPO',
 'JUZ0EVGCPO')